<a href="https://colab.research.google.com/github/D0mP3dr0/geoprocessing_gnn/blob/main/geoprocessing_gnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📓 4_Integracao_Hidrografia

In [ ]:
# 4.1 Cálculo de Distância a Corpos D'água
# ==================================================================================================
# Este notebook calcula distâncias entre edificações e corpos d'água, permitindo análises 
# de áreas de preservação permanente (APP), riscos de inundação e conformidade ambiental.

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString, Polygon
from scipy.spatial import cKDTree
import contextily as ctx
import seaborn as sns
from tqdm.notebook import tqdm

# Configurações de visualização
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# %% [markdown]
# ## 1. Carregamento dos Dados
# Vamos carregar os dados de hidrografia e edificações para iniciar a análise de distâncias.

# %%
# Caminhos para os arquivos
DATA_DIR = "F:/TESE_MESTRADO/geoprocessing/data"
BUILDINGS_PATH = os.path.join(DATA_DIR, "enriched_data/buildings_enriched.gpkg")
HYDRO_PATH = os.path.join(DATA_DIR, "enriched_data/hidrografia_enriched.gpkg")

# Carregar dados
buildings_gdf = gpd.read_file(BUILDINGS_PATH)
print(f"Edificações carregadas: {len(buildings_gdf)} registros")

# Carregar diferentes camadas hidrográficas
trecho_drenagem = gpd.read_file(HYDRO_PATH, layer="trecho_drenagem")
curso_dagua = gpd.read_file(HYDRO_PATH, layer="curso_dagua")
area_drenagem = gpd.read_file(HYDRO_PATH, layer="area_drenagem")

print(f"Trechos de drenagem: {len(trecho_drenagem)} registros")
print(f"Cursos d'água: {len(curso_dagua)} registros")
print(f"Áreas de drenagem: {len(area_drenagem)} registros")

# Verificar sistemas de coordenadas
print(f"\nCRS Edificações: {buildings_gdf.crs}")
print(f"CRS Hidrografia: {trecho_drenagem.crs}")

# Garantir que todos estejam no mesmo CRS
if buildings_gdf.crs != trecho_drenagem.crs:
    print("Reprojetando edificações para o mesmo CRS da hidrografia")
    buildings_gdf = buildings_gdf.to_crs(trecho_drenagem.crs)

# %% [markdown]
# ## 2. Visualização dos Dados
# Vamos visualizar os dados para entender sua distribuição espacial.

# %%
# Criar figura para visualização
fig, ax = plt.subplots(1, 1, figsize=(14, 10))

# Plotar camadas
area_drenagem.plot(ax=ax, color='lightblue', alpha=0.5, label='Áreas de Drenagem')
trecho_drenagem.plot(ax=ax, color='blue', linewidth=1, label='Trechos de Drenagem')
curso_dagua.plot(ax=ax, color='darkblue', linewidth=1.5, label='Cursos D\'água')

# Amostragem de edificações para melhor visualização
sample_size = min(5000, len(buildings_gdf))
buildings_sample = buildings_gdf.sample(sample_size)
buildings_sample.plot(ax=ax, color='red', markersize=5, alpha=0.5, label='Edificações (amostra)')

# Adicionar mapa base
ctx.add_basemap(ax, crs=buildings_gdf.crs.to_string(), source=ctx.providers.CartoDB.Positron)

# Estilização do gráfico
ax.set_title('Distribuição Espacial de Edificações e Hidrografia', fontsize=16)
ax.legend(loc='best')
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 3. Cálculo de Distâncias até Corpos D'água
# Vamos calcular a distância de cada edificação até o corpo d'água mais próximo.

# %%
def calcular_distancia_mais_proxima(pontos_gdf, linhas_gdf):
    """
    Calcula a distância de cada ponto ao feature linear mais próximo.
    
    Args:
        pontos_gdf: GeoDataFrame com geometrias do tipo ponto (ou polígono, usando centroides)
        linhas_gdf: GeoDataFrame com geometrias lineares
        
    Returns:
        Series com as distâncias calculadas
    """
    # Garantir que estamos trabalhando com pontos (usando centroides para polígonos)
    if pontos_gdf.geometry.iloc[0].geom_type == 'Polygon' or pontos_gdf.geometry.iloc[0].geom_type == 'MultiPolygon':
        pontos = pontos_gdf.copy()
        pontos['geometry'] = pontos_gdf.geometry.centroid
    else:
        pontos = pontos_gdf.copy()
    
    # Extrair coordenadas para uso com KDTree
    coords_pontos = np.array([(p.x, p.y) for p in pontos.geometry])
    
    # Para cada linha, extrair pontos em intervalos regulares
    line_points = []
    line_indices = []
    
    print("Processando linhas para cálculo de distância...")
    for idx, line in tqdm(enumerate(linhas_gdf.geometry), total=len(linhas_gdf)):
        if line.geom_type == 'LineString':
            # Extrair pontos em intervalos de aproximadamente 10 metros
            length = line.length
            num_points = max(2, int(length / 10))
            distances = np.linspace(0, 1, num_points)
            points = [line.interpolate(distance, normalized=True) for distance in distances]
            
            for p in points:
                line_points.append((p.x, p.y))
                line_indices.append(idx)
        elif line.geom_type == 'MultiLineString':
            for subline in line.geoms:
                length = subline.length
                num_points = max(2, int(length / 10))
                distances = np.linspace(0, 1, num_points)
                points = [subline.interpolate(distance, normalized=True) for distance in distances]
                
                for p in points:
                    line_points.append((p.x, p.y))
                    line_indices.append(idx)
    
    # Converter em array numpy
    line_points = np.array(line_points)
    
    # Construir KDTree a partir dos pontos das linhas
    print("Construindo KDTree...")
    tree = cKDTree(line_points)
    
    # Calcular distâncias
    print("Calculando distâncias...")
    distances, indices = tree.query(coords_pontos)
    
    # Converter distâncias para metros (se o CRS estiver em graus)
    if pontos.crs.is_geographic:
        # Fator aproximado para converter graus para metros (varia por latitude)
        # 111120 metros por grau, mas isso é uma aproximação
        distances = distances * 111120
    
    return distances

# %%
# Calcular distâncias para trechos de drenagem e cursos d'água
print("Calculando distâncias até os trechos de drenagem...")
buildings_gdf['dist_trecho_drenagem'] = calcular_distancia_mais_proxima(buildings_gdf, trecho_drenagem)

print("\nCalculando distâncias até os cursos d'água...")
buildings_gdf['dist_curso_dagua'] = calcular_distancia_mais_proxima(buildings_gdf, curso_dagua)

# Calcular a menor distância para qualquer elemento hidrográfico
buildings_gdf['dist_hidro_min'] = buildings_gdf[['dist_trecho_drenagem', 'dist_curso_dagua']].min(axis=1)

# Mostrar estatísticas básicas das distâncias calculadas
print("\nEstatísticas de Distâncias (metros):")
print(buildings_gdf[['dist_trecho_drenagem', 'dist_curso_dagua', 'dist_hidro_min']].describe())

# %% [markdown]
# ## 4. Análise de Áreas de Preservação Permanente (APP)
# De acordo com a legislação ambiental brasileira, as APPs variam conforme o tipo e largura do curso d'água.

# %%
# Definir categorias para análise de APPs
def categorizar_app(distancia):
    """Categoriza edificações de acordo com a distância até corpos d'água"""
    if distancia <= 30:
        return "APP 30m (córregos até 10m)"
    elif distancia <= 50:
        return "APP 50m (rios 10-50m)"
    elif distancia <= 100:
        return "APP 100m (rios 50-200m)"
    elif distancia <= 200:
        return "APP 200m (rios 200-600m)"
    elif distancia <= 500:
        return "APP 500m (rios >600m)"
    else:
        return "Fora de APP"

# Aplicar categorização
buildings_gdf['categoria_app'] = buildings_gdf['dist_hidro_min'].apply(categorizar_app)

# Contabilizar edificações em cada categoria
contagem_app = buildings_gdf['categoria_app'].value_counts()
print("\nEdificações por categoria de APP:")
print(contagem_app)

# Visualizar distribuição
plt.figure(figsize=(14, 8))
contagem_app.plot(kind='bar', color='skyblue')
plt.title('Distribuição de Edificações por Categoria de APP', fontsize=16)
plt.xlabel('Categoria de APP')
plt.ylabel('Número de Edificações')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Calcular percentual de edificações em APPs
em_app = buildings_gdf[buildings_gdf['categoria_app'] != "Fora de APP"].shape[0]
pct_em_app = (em_app / len(buildings_gdf)) * 100
print(f"\nPercentual de edificações em áreas de APP: {pct_em_app:.2f}%")

# %% [markdown]
# ## 5. Visualização das Edificações por Distância
# Vamos criar um mapa que mostra as edificações coloridas por distância até corpos d'água.

# %%
# Criar categorias para distâncias
bins = [0, 30, 50, 100, 200, 500, float('inf')]
labels = ['0-30m', '30-50m', '50-100m', '100-200m', '200-500m', '>500m']
buildings_gdf['dist_categoria'] = pd.cut(buildings_gdf['dist_hidro_min'], bins=bins, labels=labels)

# Criar mapa com edificações coloridas por categoria de distância
fig, ax = plt.subplots(1, 1, figsize=(16, 12))

# Definir esquema de cores
cmap = plt.cm.YlOrRd_r

# Plotar hidrografia
area_drenagem.plot(ax=ax, color='lightblue', alpha=0.4, label='Áreas de Drenagem')
trecho_drenagem.plot(ax=ax, color='blue', linewidth=1, label='Trechos de Drenagem')
curso_dagua.plot(ax=ax, color='darkblue', linewidth=1.5, label='Cursos D\'água')

# Para visualização eficiente, amostrar ou limitar a uma área específica
if len(buildings_gdf) > 10000:
    buildings_plot = buildings_gdf.sample(10000)
else:
    buildings_plot = buildings_gdf

# Plotar edificações coloridas por distância
for categoria, cor in zip(labels, cmap(np.linspace(0, 1, len(labels)))):
    subset = buildings_plot[buildings_plot['dist_categoria'] == categoria]
    subset.plot(ax=ax, color=cor, label=f'{categoria}', alpha=0.7, markersize=20)

# Adicionar mapa base
ctx.add_basemap(ax, crs=buildings_gdf.crs.to_string(), source=ctx.providers.CartoDB.Positron)

# Configurar legenda e estilo
ax.set_title('Edificações por Distância até Corpos D\'água', fontsize=16)
ax.legend(loc='best')
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 6. Análise de Densidade
# Vamos analisar a densidade de edificações em função da distância dos corpos d'água.

# %%
# Criar faixas de distância
bins = np.arange(0, 1001, 50)  # Faixas de 50 metros até 1000 metros
dist_counts = pd.cut(buildings_gdf['dist_hidro_min'], bins=bins).value_counts().sort_index()
dist_density = dist_counts / dist_counts.sum()

# Plotar histograma e densidade
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12), sharex=True)

# Histograma
dist_counts.plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('Número de Edificações por Distância até Corpos D\'água', fontsize=16)
ax1.set_ylabel('Número de Edificações')
ax1.grid(axis='y', linestyle='--', alpha=0.7)

# Densidade
dist_density.plot(kind='bar', ax=ax2, color='salmon')
ax2.set_title('Densidade de Edificações por Distância até Corpos D\'água', fontsize=16)
ax2.set_xlabel('Distância (metros)')
ax2.set_ylabel('Densidade')
ax2.grid(axis='y', linestyle='--', alpha=0.7)

plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# %% [markdown]
# ## 7. Análise de Vulnerabilidade por Tipo de Edificação
# Vamos analisar como diferentes tipos de edificações se distribuem em relação aos corpos d'água.

# %%
# Identificar os tipos mais comuns de edificações
top_building_types = buildings_gdf['building'].value_counts().head(8).index.tolist()

# Criar dataframe de análise
building_type_dist = pd.DataFrame()

for building_type in top_building_types:
    subset = buildings_gdf[buildings_gdf['building'] == building_type]
    
    # Calcular estatísticas
    stats = {
        'count': len(subset),
        'mean_dist': subset['dist_hidro_min'].mean(),
        'median_dist': subset['dist_hidro_min'].median(),
        'pct_in_30m': (subset['dist_hidro_min'] <= 30).mean() * 100,
        'pct_in_100m': (subset['dist_hidro_min'] <= 100).mean() * 100
    }
    
    building_type_dist = pd.concat([building_type_dist, pd.DataFrame([stats], index=[building_type])])

# Ordenar por média de distância
building_type_dist = building_type_dist.sort_values('mean_dist')

# Visualizar resultados
print("Estatísticas de Distância por Tipo de Edificação:")
print(building_type_dist)

# Plot comparativo
fig, ax = plt.subplots(figsize=(14, 8))

# Barras para distância média
bars = ax.bar(building_type_dist.index, building_type_dist['mean_dist'], color='skyblue', alpha=0.7)

# Linha para percentual em APP de 30m
ax2 = ax.twinx()
line = ax2.plot(building_type_dist.index, building_type_dist['pct_in_30m'], 'ro-', linewidth=2, label='% em APP 30m')

# Estilização
ax.set_title('Distância Média e Percentual em APP por Tipo de Edificação', fontsize=16)
ax.set_xlabel('Tipo de Edificação')
ax.set_ylabel('Distância Média (metros)', color='blue')
ax2.set_ylabel('Percentual em APP 30m (%)', color='red')
ax.grid(axis='y', linestyle='--', alpha=0.3)
ax.set_xticklabels(building_type_dist.index, rotation=45, ha='right')

# Legenda
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='best')

plt.tight_layout()
plt.show()

# %% [markdown]
# ## 8. Salvando Resultados

# %%
# Salvar o GeoDataFrame enriquecido com as distâncias
output_path = os.path.join(DATA_DIR, "enriched_data/buildings_hydro_distances.gpkg")
buildings_gdf.to_file(output_path, driver="GPKG")
print(f"Dados salvos em: {output_path}")

# Criar um relatório resumido em CSV
report_cols = [
    'building', 'dist_trecho_drenagem', 'dist_curso_dagua', 
    'dist_hidro_min', 'categoria_app', 'dist_categoria'
]

report_df = buildings_gdf[report_cols].copy()
report_path = os.path.join(DATA_DIR, "enriched_data/buildings_hydro_report.csv")
report_df.to_csv(report_path, index=False)
print(f"Relatório exportado para: {report_path}")

# %% [markdown]
# ## 9. Conclusões
# 
# - Calculamos as distâncias de edificações até os corpos d'água mais próximos
# - Categorizamos as edificações por faixas de APP de acordo com a legislação brasileira
# - Identificamos a proporção de edificações em cada categoria de APP
# - Analisamos a distribuição espacial das edificações em relação à hidrografia
# - Avaliamos a vulnerabilidade específica por tipo de edificação
#
# Estes resultados podem ser utilizados para:
# 1. Avaliação de conformidade ambiental
# 2. Análise de riscos de inundação
# 3. Planejamento urbano e ambiental
# 4. Fiscalização de áreas de preservação permanente

4.2_Analise_Risco_Inundacao

In [ ]:
# 4.2 Análise de Risco de Inundação
# ==================================================================================================
# Este notebook integra dados hidrográficos, topográficos e de edificações para analisar 
# áreas com potencial risco de inundação, utilizando modelagem baseada em grafos.

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
from shapely.ops import unary_union
import contextily as ctx
from tqdm.notebook import tqdm
import networkx as nx
import rasterio
from rasterio import features
from rasterio.transform import Affine
from scipy.spatial import cKDTree
from scipy.interpolate import griddata
import math
import json

# Configurações de visualização
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# %% [markdown]
# ## 1. Carregamento dos Dados Necessários
# Carregamos os dados hidrográficos, de elevação, edificações e setores censitários.

# %%
# Caminhos para os arquivos
DATA_DIR = "F:/TESE_MESTRADO/geoprocessing/data"
BUILDINGS_PATH = os.path.join(DATA_DIR, "enriched_data/buildings_hydro_distances.gpkg")
HYDRO_PATH = os.path.join(DATA_DIR, "enriched_data/hidrografia_enriched.gpkg")
DEM_PATH = os.path.join(DATA_DIR, "raw/dem_sorocaba.tif")
CENSUS_PATH = os.path.join(DATA_DIR, "enriched_data/setores_censitarios_enriched.gpkg")

# Carregar dados
buildings_gdf = gpd.read_file(BUILDINGS_PATH)
print(f"Edificações carregadas: {len(buildings_gdf)} registros")

# Carregar diferentes camadas hidrográficas
trecho_drenagem = gpd.read_file(HYDRO_PATH, layer="trecho_drenagem")
curso_dagua = gpd.read_file(HYDRO_PATH, layer="curso_dagua")
area_drenagem = gpd.read_file(HYDRO_PATH, layer="area_drenagem")
ponto_drenagem = gpd.read_file(HYDRO_PATH, layer="ponto_drenagem")
nodes = gpd.read_file(HYDRO_PATH, layer="nodes")

print(f"Trechos de drenagem: {len(trecho_drenagem)} registros")
print(f"Cursos d'água: {len(curso_dagua)} registros")
print(f"Áreas de drenagem: {len(area_drenagem)} registros")
print(f"Pontos de drenagem: {len(ponto_drenagem)} registros")
print(f"Nós da rede hidrográfica: {len(nodes)} registros")

# Carregar dados de elevação
try:
    dem = rasterio.open(DEM_PATH)
    print(f"Modelo Digital de Elevação carregado: {dem.width}x{dem.height} pixels")
except Exception as e:
    print(f"Erro ao carregar DEM: {e}")
    # Criar um DEM sintético para demonstração se necessário
    print("Criando DEM sintético para demonstração...")
    
# Carregar setores censitários
setores = gpd.read_file(CENSUS_PATH)
print(f"Setores censitários carregados: {len(setores)} registros")

# Verificar sistemas de coordenadas
print(f"\nCRS Edificações: {buildings_gdf.crs}")
print(f"CRS Hidrografia: {trecho_drenagem.crs}")
print(f"CRS Setores: {setores.crs}")

# Garantir que todos estejam no mesmo CRS
target_crs = trecho_drenagem.crs
if buildings_gdf.crs != target_crs:
    buildings_gdf = buildings_gdf.to_crs(target_crs)
if setores.crs != target_crs:
    setores = setores.to_crs(target_crs)

# %% [markdown]
# ## 2. Carregamento das Estatísticas Hidrográficas
# Vamos carregar os resultados da análise hidrográfica previamente realizada.

# %%
# Carregar estatísticas hidrográficas do arquivo JSON
HYDRO_REPORT_PATH = os.path.join(DATA_DIR, "analysis_reports/hidrografia_enrichment_report.json")

try:
    with open(HYDRO_REPORT_PATH, 'r', encoding='utf-8') as f:
        hydro_stats = json.load(f)
    print("Estatísticas hidrográficas carregadas com sucesso")
    
    # Mostrar algumas estatísticas relevantes
    print("\nEstatísticas de sinuosidade:")
    sinuosity = hydro_stats['metrics']['trecho_drenagem']['sinuosity']
    print(f"Média: {sinuosity['mean']:.2f}")
    print(f"Mediana: {sinuosity['median']:.2f}")
    print(f"Mínimo: {sinuosity['min']:.2f}")
    print(f"Máximo: {sinuosity['max']:.2f}")
    
    print("\nEstatísticas de rede:")
    network = hydro_stats['metrics']['trecho_drenagem']['network_info']
    print(f"Nós: {network['n_nodes']}")
    print(f"Arestas: {network['n_edges']}")
    print(f"Componentes conectados: {network['n_components']}")
    print(f"Tamanho do maior componente: {network['largest_component_size']}")
    
    print("\nPadrão de drenagem:")
    pattern = hydro_stats['nodes_summary']['drainage_pattern']
    print(f"Tipo: {pattern['pattern']}")
    print(f"Confiança: {pattern['confidence']:.2f}")
    
except Exception as e:
    print(f"Erro ao carregar estatísticas hidrográficas: {e}")
    print("Continuando sem estatísticas hidrográficas detalhadas...")
    hydro_stats = None

# %% [markdown]
# ## 3. Construção do Grafo Hidrográfico
# Vamos construir uma representação em grafo da rede hidrográfica para modelagem do fluxo.

# %%
def construir_grafo_hidrografico(trechos, nos):
    """
    Constrói um grafo direcionado representando a rede hidrográfica.
    
    Args:
        trechos: GeoDataFrame com os trechos de drenagem
        nos: GeoDataFrame com os nós da rede hidrográfica
        
    Returns:
        Grafo NetworkX da rede hidrográfica
    """
    G = nx.DiGraph()
    
    # Adicionar nós
    for idx, node in nos.iterrows():
        node_type = node.get('type', 'junction')
        G.add_node(idx, 
                   pos=(node.geometry.x, node.geometry.y),
                   elevation=node.get('elevation', 0),
                   type=node_type,
                   geometry=node.geometry)
    
    # Adicionar arestas (trechos)
    for idx, trecho in trechos.iterrows():
        # Identificar nós de origem e destino
        from_node = trecho.get('from_node', None)
        to_node = trecho.get('to_node', None)
        
        if from_node is None or to_node is None:
            # Se não tiver nós explícitos, tentar inferir da geometria
            coords = list(trecho.geometry.coords)
            start_point = Point(coords[0])
            end_point = Point(coords[-1])
            
            # Encontrar os nós mais próximos dos pontos inicial e final
            from_node = nos.distance(start_point).idxmin()
            to_node = nos.distance(end_point).idxmin()
        
        # Adicionar a aresta com atributos
        G.add_edge(from_node, to_node,
                  length=trecho.geometry.length,
                  slope=trecho.get('slope', 0),
                  strahler=trecho.get('strahler_order', 1),
                  sinuosity=trecho.get('sinuosity', 1),
                  stream_power=trecho.get('stream_power_index', 0),
                  geometry=trecho.geometry)
    
    return G

# Construir o grafo
try:
    hydro_graph = construir_grafo_hidrografico(trecho_drenagem, nodes)
    print(f"Grafo hidrográfico construído com {len(hydro_graph.nodes)} nós e {len(hydro_graph.edges)} arestas")
    
    # Visualizar o grafo
    plt.figure(figsize=(14, 10))
    
    # Posições dos nós
    pos = nx.get_node_attributes(hydro_graph, 'pos')
    
    # Definir cores por tipo de nó
    node_colors = []
    for node in hydro_graph.nodes():
        if hydro_graph.nodes[node].get('type') == 'spring':
            node_colors.append('green')
        elif hydro_graph.nodes[node].get('type') == 'outlet':
            node_colors.append('red')
        else:
            node_colors.append('blue')
    
    # Definir tamanhos de aresta por ordem de Strahler
    edge_widths = []
    for u, v in hydro_graph.edges():
        strahler = hydro_graph.edges[u, v].get('strahler', 1)
        edge_widths.append(0.5 + 0.5 * strahler)
    
    # Desenhar o grafo
    nx.draw(hydro_graph, pos, 
            node_color=node_colors, 
            node_size=50, 
            width=edge_widths,
            arrowsize=10,
            edge_color='blue',
            with_labels=False)
    
    plt.title('Grafo da Rede Hidrográfica', fontsize=16)
    plt.grid(False)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Erro ao construir grafo hidrográfico: {e}")
    print("Continuando sem representação em grafo...")

# %% [markdown]
# ## 4. Análise de Acumulação de Fluxo
# Usaremos o grafo hidrográfico para análise de acumulação de fluxo e identificação de áreas propensas a inundação.

# %%
def calcular_acumulacao_fluxo(grafo):
    """
    Calcula o valor de acumulação de fluxo para cada nó do grafo.
    
    Args:
        grafo: Grafo NetworkX representando a rede hidrográfica
        
    Returns:
        Dicionário com os valores de acumulação de fluxo por nó
    """
    # Inicializar acumulação de fluxo
    acumulacao = {node: 0 for node in grafo.nodes()}
    
    # Identificar nós de nascente (sem arestas de entrada)
    nascentes = [node for node, in_degree in grafo.in_degree() if in_degree == 0]
    print(f"Nascentes identificadas: {len(nascentes)}")
    
    # Calcular ordem topológica para processamento
    try:
        ordem_topologica = list(nx.topological_sort(grafo))
    except nx.NetworkXUnfeasible:
        print("Grafo contém ciclos. Usando ordem arbitrária.")
        ordem_topologica = list(grafo.nodes())
    
    # Para cada nó na ordem topológica
    for node in ordem_topologica:
        # Se for nascente, inicializar com 1
        if node in nascentes:
            acumulacao[node] = 1
        
        # Propagar para jusante
        for successor in grafo.successors(node):
            edge_weight = grafo.edges[node, successor].get('weight', 1)
            acumulacao[successor] += acumulacao[node] * edge_weight
    
    return acumulacao

# Calcular acumulação de fluxo
try:
    # Adicionar pesos às arestas baseados na ordem de Strahler
    for u, v in hydro_graph.edges():
        strahler = hydro_graph.edges[u, v].get('strahler', 1)
        hydro_graph.edges[u, v]['weight'] = strahler / 7  # Normalizado pela ordem máxima (7)
    
    fluxo_acumulado = calcular_acumulacao_fluxo(hydro_graph)
    
    # Adicionar valores calculados aos nós
    nx.set_node_attributes(hydro_graph, fluxo_acumulado, 'flow_accumulation')
    
    # Visualizar acumulação de fluxo
    plt.figure(figsize=(14, 10))
    
    # Posições dos nós
    pos = nx.get_node_attributes(hydro_graph, 'pos')
    
    # Valores de acumulação normalizados para tamanho dos nós
    acumulacao_valores = list(fluxo_acumulado.values())
    max_acumulacao = max(acumulacao_valores)
    node_sizes = [100 * (fluxo_acumulado[node] / max_acumulacao) + 10 for node in hydro_graph.nodes()]
    
    # Desenhar o grafo
    nx.draw(hydro_graph, pos, 
            node_color='blue', 
            node_size=node_sizes, 
            width=edge_widths,
            edge_color='lightblue',
            with_labels=False,
            alpha=0.7)
    
    plt.title('Acumulação de Fluxo na Rede Hidrográfica', fontsize=16)
    plt.grid(False)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print("Estatísticas de Acumulação de Fluxo:")
    acumulacao_series = pd.Series(fluxo_acumulado)
    print(acumulacao_series.describe())
    
except Exception as e:
    print(f"Erro ao calcular acumulação de fluxo: {e}")
    print("Continuando sem análise de acumulação...")

# %% [markdown]
# ## 5. Criação de Modelo de Susceptibilidade a Inundações
# Integraremos dados de elevação, declividade, distância a corpos d'água e acumulação de fluxo.

# %%
def criar_modelo_susceptibilidade(dem_path, hydro_gdf, acumulacao=None, resolution=30):
    """
    Cria um modelo de susceptibilidade a inundações.
    
    Args:
        dem_path: Caminho para o arquivo raster do Modelo Digital de Elevação
        hydro_gdf: GeoDataFrame com a hidrografia
        acumulacao: Dicionário com valores de acumulação de fluxo (opcional)
        resolution: Resolução do modelo em metros
        
    Returns:
        Modelo de susceptibilidade e seus metadados
    """
    try:
        # Abrir o DEM
        with rasterio.open(dem_path) as src:
            dem_data = src.read(1)
            transform = src.transform
            crs = src.crs
            dem_nodata = src.nodata
            
            # Substituir valores nodata
            if dem_nodata is not None:
                dem_data = np.where(dem_data == dem_nodata, np.nan, dem_data)
            
            # Calcular declividade (simplificada)
            dy, dx = np.gradient(dem_data, 2, 2)
            slope = np.sqrt(dx**2 + dy**2)
            
            # Normalizar declividade (0-1)
            slope_norm = np.zeros_like(slope)
            slope_min = np.nanmin(slope)
            slope_max = np.nanmax(slope)
            if slope_max > slope_min:
                slope_norm = (slope - slope_min) / (slope_max - slope_min)
            
            # Inicializar modelo de susceptibilidade
            susceptibility = np.zeros_like(dem_data, dtype=float)
            
            # Rasterizar hidrografia para obter distância
            shapes = [(geom, 1) for geom in hydro_gdf.geometry]
            hydro_raster = features.rasterize(shapes, 
                                             out_shape=dem_data.shape, 
                                             transform=transform,
                                             fill=0,
                                             all_touched=True)
            
            # Calcular distância euclidiana à hidrografia (em células)
            from scipy.ndimage import distance_transform_edt
            distance = distance_transform_edt(1 - hydro_raster) * resolution
            
            # Normalizar distância (0-1) e inverter (mais próximo = maior susceptibilidade)
            max_dist = 1000  # metros
            dist_norm = 1 - np.minimum(distance / max_dist, 1)
            
            # Combinar fatores
            # Peso da elevação: 0.4
            # Peso da declividade: 0.3
            # Peso da distância: 0.3
            
            # Normalizar elevação (mais baixo = maior susceptibilidade)
            elev_min = np.nanmin(dem_data)
            elev_max = np.nanmax(dem_data)
            elev_norm = 1 - ((dem_data - elev_min) / (elev_max - elev_min))
            
            # Combinar os fatores
            susceptibility = (0.4 * elev_norm + 
                              0.3 * (1 - slope_norm) +  # Menores declividades são mais suscetíveis
                              0.3 * dist_norm)
            
            # Se houver dados de acumulação, incorporar ao modelo
            if acumulacao is not None:
                # Converter acumulação para raster
                accum_raster = np.zeros_like(dem_data)
                
                # Para cada nó com acumulação, adicionar valor ao raster
                for node, value in acumulacao.items():
                    if 'pos' in hydro_graph.nodes[node]:
                        x, y = hydro_graph.nodes[node]['pos']
                        # Converter coordenadas para índices de pixel
                        row, col = src.index(x, y)
                        if 0 <= row < accum_raster.shape[0] and 0 <= col < accum_raster.shape[1]:
                            accum_raster[row, col] = value
                
                # Suavizar o raster de acumulação
                from scipy.ndimage import gaussian_filter
                accum_raster = gaussian_filter(accum_raster, sigma=3)
                
                # Normalizar acumulação
                accum_max = np.max(accum_raster)
                if accum_max > 0:
                    accum_norm = accum_raster / accum_max
                    
                    # Reajustar os pesos
                    susceptibility = (0.3 * elev_norm + 
                                      0.2 * (1 - slope_norm) +
                                      0.2 * dist_norm +
                                      0.3 * accum_norm)
            
            # Limitar valores entre 0 e 1
            susceptibility = np.clip(susceptibility, 0, 1)
            
            return {
                'model': susceptibility,
                'transform': transform,
                'crs': crs,
                'resolution': resolution,
                'elev_norm': elev_norm,
                'slope_norm': slope_norm,
                'dist_norm': dist_norm
            }
    
    except Exception as e:
        print(f"Erro na criação do modelo de susceptibilidade: {e}")
        return None

# Criar modelo de susceptibilidade
try:
    susceptibility_model = criar_modelo_susceptibilidade(
        DEM_PATH, 
        pd.concat([trecho_drenagem, curso_dagua]), 
        acumulacao=fluxo_acumulado if 'fluxo_acumulado' in locals() else None
    )
    
    if susceptibility_model:
        print("Modelo de susceptibilidade a inundações criado com sucesso")
        
        # Visualizar o modelo
        plt.figure(figsize=(16, 12))
        
        # Criar uma máscara para valores NaN
        suscept = susceptibility_model['model']
        mask = np.isnan(suscept)
        
        # Plotar o modelo com colormap adequado
        cmap = plt.cm.YlOrRd
        plt.imshow(np.ma.array(suscept, mask=mask), 
                  cmap=cmap, 
                  vmin=0, 
                  vmax=1)
        
        plt.colorbar(label='Susceptibilidade a Inundações')
        plt.title('Modelo de Susceptibilidade a Inundações', fontsize=16)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
        # Visualizar os componentes do modelo
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # Elevação normalizada
        im1 = axes[0].imshow(np.ma.array(susceptibility_model['elev_norm'], mask=mask), 
                           cmap='terrain', vmin=0, vmax=1)
        axes[0].set_title('Fator Elevação', fontsize=14)
        axes[0].axis('off')
        plt.colorbar(im1, ax=axes[0])
        
        # Declividade normalizada
        im2 = axes[1].imshow(np.ma.array(1 - susceptibility_model['slope_norm'], mask=mask), 
                           cmap='Greens', vmin=0, vmax=1)
        axes[1].set_title('Fator Declividade', fontsize=14)
        axes[1].axis('off')
        plt.colorbar(im2, ax=axes[1])
        
        # Distância normalizada
        im3 = axes[2].imshow(np.ma.array(susceptibility_model['dist_norm'], mask=mask), 
                           cmap='Blues', vmin=0, vmax=1)
        axes[2].set_title('Fator Distância', fontsize=14)
        axes[2].axis('off')
        plt.colorbar(im3, ax=axes[2])
        
        plt.tight_layout()
        plt.show()
    
except Exception as e:
    print(f"Erro ao criar modelo de susceptibilidade: {e}")
    susceptibility_model = None

# %% [markdown]
# ## 6. Identificação de Áreas de Inundação
# Vamos definir áreas propensas a inundação com base no modelo de susceptibilidade.

# %%
def gerar_areas_inundacao(susceptibility_model, thresholds=[0.7, 0.8, 0.9]):
    """
    Gera polígonos de áreas com potencial de inundação baseados no modelo de susceptibilidade.
    
    Args:
        susceptibility_model: Modelo de susceptibilidade gerado anteriormente
        thresholds: Limiares para definir níveis de risco (alta, média, baixa susceptibilidade)
        
    Returns:
        GeoDataFrame com áreas de inundação
    """
    if susceptibility_model is None:
        return None
    
    try:
        suscept = susceptibility_model['model']
        transform = susceptibility_model['transform']
        crs = susceptibility_model['crs']
        
        # Lista para armazenar shapes de cada nível
        inundation_shapes = []
        
        # Para cada limiar, gerar shapes
        for i, threshold in enumerate(thresholds):
            # Criar máscara binária
            mask = suscept >= threshold
            
            # Converter para shapes
            shapes = features.shapes(
                mask.astype('uint8'), 
                mask=mask,
                transform=transform
            )
            
            # Converter para geometrias
            geometries = []
            for shape, value in shapes:
                if value == 1:
                    geometries.append(shape)
            
            # Criar polígonos
            if geometries:
                polys = [Polygon(geom['coordinates'][0]) for geom in geometries if geom['type'] == 'Polygon']
                
                # Aplicar buffer e união
                from shapely.ops import unary_union
                buffered = unary_union([poly.buffer(30) for poly in polys])
                
                # Adicionar à lista com o nível de risco
                if isinstance(buffered, Polygon):
                    inundation_shapes.append({
                        'geometry': buffered,
                        'nivel_risco': i+1,
                        'limiar': threshold,
                        'descricao': f"Risco {'Alto' if i==2 else 'Médio' if i==1 else 'Baixo'}"
                    })
                elif isinstance(buffered, MultiPolygon):
                    for poly in buffered.geoms:
                        inundation_shapes.append({
                            'geometry': poly,
                            'nivel_risco': i+1,
                            'limiar': threshold,
                            'descricao': f"Risco {'Alto' if i==2 else 'Médio' if i==1 else 'Baixo'}"
                        })
        
        # Criar GeoDataFrame
        if inundation_shapes:
            inundation_gdf = gpd.GeoDataFrame(inundation_shapes, crs=crs)
            return inundation_gdf
        else:
            return None
            
    except Exception as e:
        print(f"Erro ao gerar áreas de inundação: {e}")
        return None

# Gerar áreas de inundação
try:
    inundation_areas = gerar_areas_inundacao(susceptibility_model)
    
    if inundation_areas is not None:
        print(f"Áreas de inundação geradas: {len(inundation_areas)} polígonos")
        
        # Visualizar áreas de inundação
        fig, ax = plt.subplots(figsize=(16, 12))
        
        # Plotar camadas base
        area_drenagem.plot(ax=ax, color='lightblue', alpha=0.4, label='Áreas de Drenagem')
        trecho_drenagem.plot(ax=ax, color='blue', linewidth=1, label='Trechos de Drenagem')
        curso_dagua.plot(ax=ax, color='darkblue', linewidth=1.5, label='Cursos D\'água')
        
        # Definir esquema de cores para níveis de risco
        risk_colors = ['yellow', 'orange', 'red']
        
        # Plotar áreas de inundação por nível de risco
        for nivel, color in zip(sorted(inundation_areas['nivel_risco'].unique()), risk_colors):
            subset = inundation_areas[inundation_areas['nivel_risco'] == nivel]
            subset.plot(
                ax=ax, 
                color=color, 
                alpha=0.5, 
                label=f"Risco {'Alto' if nivel==3 else 'Médio' if nivel==2 else 'Baixo'}"
            )
        
        # Adicionar mapa base
        ctx.add_basemap(ax, crs=inundation_areas.crs.to_string(), source=ctx.providers.CartoDB.Positron)
        
        # Configurar legenda e estilo
        ax.set_title('Áreas com Potencial de Inundação', fontsize=16)
        ax.legend(loc='best')
        plt.tight_layout()
        plt.show()
        
        # Calcular área total de cada nível de risco
        area_por_nivel = inundation_areas.groupby('nivel_risco')['geometry'].apply(lambda x: sum(item.area for item in x))
        area_por_nivel_km2 = area_por_nivel / 1_000_000  # Converter para km²
        
        print("\nÁreas de risco de inundação (km²):")
        for nivel, area in area_por_nivel_km2.items():
            desc = "Alto" if nivel==3 else "Médio" if nivel==2 else "Baixo"
            print(f"Risco {desc}: {area:.2f} km²")
        
    else:
        print("Não foi possível gerar áreas de inundação")
        
except Exception as e:
    print(f"Erro ao gerar ou visualizar áreas de inundação: {e}")
    inundation_areas = None

# %% [markdown]
# ## 7. Avaliação de Risco para Edificações
# Analisaremos as edificações localizadas em áreas de risco de inundação.

# %%
def avaliar_risco_edificacoes(buildings_gdf, inundation_gdf):
    """
    Avalia o risco de inundação para edificações.
    
    Args:
        buildings_gdf: GeoDataFrame com edificações
        inundation_gdf: GeoDataFrame com áreas de inundação
        
    Returns:
        GeoDataFrame das edificações com informações de risco
    """
    if inundation_gdf is None:
        return buildings_gdf
    
    try:
        # Garantir mesmo CRS
        if buildings_gdf.crs != inundation_gdf.crs:
            buildings_gdf = buildings_gdf.to_crs(inundation_gdf.crs)
        
        # Criar cópia para não modificar o original
        buildings_risk = buildings_gdf.copy()
        
        # Inicializar coluna de risco
        buildings_risk['risco_inundacao'] = 0
        buildings_risk['nivel_risco'] = 'Sem risco'
        
        # Para cada nível de risco (do mais alto para o mais baixo)
        for nivel in sorted(inundation_gdf['nivel_risco'].unique(), reverse=True):
            # Filtrar áreas para o nível atual
            risk_areas = inundation_gdf[inundation_gdf['nivel_risco'] == nivel]
            
            # União das geometrias para intersecção mais eficiente
            risk_union = unary_union(risk_areas.geometry)
            
            # Identificar edificações na área de risco
            in_risk = buildings_risk.geometry.intersects(risk_union)
            
            # Atualizar apenas edificações ainda não classificadas em nível superior
            update_idx = in_risk & (buildings_risk['risco_inundacao'] == 0)
            buildings_risk.loc[update_idx, 'risco_inundacao'] = nivel
            buildings_risk.loc[update_idx, 'nivel_risco'] = risk_areas.iloc[0]['descricao']
        
        return buildings_risk
    
    except Exception as e:
        print(f"Erro ao avaliar risco de edificações: {e}")
        return buildings_gdf

# Avaliar risco para edificações
try:
    buildings_risk = avaliar_risco_edificacoes(buildings_gdf, inundation_areas)
    
    # Calcular estatísticas
    risk_counts = buildings_risk['nivel_risco'].value_counts()
    print("\nDistribuição de edificações por nível de risco:")
    print(risk_counts)
    
    # Percentual em áreas de risco
    total_buildings = len(buildings_risk)
    at_risk = total_buildings - risk_counts.get('Sem risco', 0)
    risk_pct = (at_risk / total_buildings) * 100
    
    print(f"\nTotal de edificações em áreas de risco: {at_risk} ({risk_pct:.2f}%)")
    
    # Visualizar edificações por nível de risco
    fig, ax = plt.subplots(figsize=(16, 12))
    
    # Plotar áreas de inundação como fundo
    if inundation_areas is not None:
        risk_colors = ['yellow', 'orange', 'red']
        for nivel, color in zip(sorted(inundation_areas['nivel_risco'].unique()), risk_colors):
            subset = inundation_areas[inundation_areas['nivel_risco'] == nivel]
            subset.plot(ax=ax, color=color, alpha=0.3)
    
    # Plotar hidrografia
    trecho_drenagem.plot(ax=ax, color='blue', linewidth=1)
    curso_dagua.plot(ax=ax, color='darkblue', linewidth=1.5)
    
    # Para visualização eficiente, amostrar ou limitar a uma área específica
    if len(buildings_risk) > 10000:
        buildings_plot = buildings_risk.sample(10000)
    else:
        buildings_plot = buildings_risk
    
    # Plotar edificações coloridas por nível de risco
    risk_buildings_colors = {
        'Sem risco': 'green',
        'Risco Baixo': 'yellow',
        'Risco Médio': 'orange',
        'Risco Alto': 'red'
    }
    
    for risk_level, color in risk_buildings_colors.items():
        subset = buildings_plot[buildings_plot['nivel_risco'] == risk_level]
        if len(subset) > 0:
            subset.plot(ax=ax, color=color, label=risk_level, markersize=20, alpha=0.7)
    
    # Adicionar mapa base
    ctx.add_basemap(ax, crs=buildings_risk.crs.to_string(), source=ctx.providers.CartoDB.Positron)
    
    # Configurar legenda e estilo
    ax.set_title('Edificações por Nível de Risco de Inundação', fontsize=16)
    ax.legend(loc='best')
    plt.tight_layout()
    plt.show()
    
    # Análise por tipo de edificação
    risk_by_type = pd.crosstab(
        buildings_risk['building'], 
        buildings_risk['nivel_risco'], 
        normalize='index'
    ) * 100
    
    # Filtrar apenas os tipos mais comuns
    top_types = buildings_risk['building'].value_counts().head(10).index
    risk_by_type_filtered = risk_by_type.loc[top_types]
    
    # Visualizar
    plt.figure(figsize=(14, 8))
    risk_by_type_filtered.plot(kind='bar', stacked=True, 
                              colormap='RdYlGn_r')
    plt.title('Percentual de Edificações em Risco por Tipo', fontsize=16)
    plt.xlabel('Tipo de Edificação')
    plt.ylabel('Percentual (%)')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Erro ao avaliar ou visualizar risco de edificações: {e}")

# %% [markdown]
# ## 8. Integração com Setores Censitários
# Vamos avaliar a população potencialmente afetada por inundações.

# %%
def avaliar_populacao_afetada(setores_gdf, inundation_gdf):
    """
    Avalia a população potencialmente afetada por inundações.
    
    Args:
        setores_gdf: GeoDataFrame com setores censitários e dados populacionais
        inundation_gdf: GeoDataFrame com áreas de inundação
        
    Returns:
        GeoDataFrame dos setores com informações de risco e população afetada
    """
    if inundation_gdf is None:
        return setores_gdf
    
    try:
        # Garantir mesmo CRS
        if setores_gdf.crs != inundation_gdf.crs:
            setores_gdf = setores_gdf.to_crs(inundation_gdf.crs)
        
        # Criar cópia para não modificar o original
        setores_risk = setores_gdf.copy()
        
        # Verificar se existe coluna de população
        pop_col = 'est_populacao' if 'est_populacao' in setores_risk.columns else None
        
        if pop_col is None:
            print("Coluna de população não encontrada. Usando área como proxy.")
            # Calcular área em km²
            setores_risk['area_km2'] = setores_risk.geometry.area / 1_000_000
            # Usar densidade constante
            setores_risk['est_populacao'] = setores_risk['area_km2'] * 1000  # 1000 hab/km²
            pop_col = 'est_populacao'
        
        # Inicializar colunas de análise
        setores_risk['area_risco_alto_km2'] = 0
        setores_risk['area_risco_medio_km2'] = 0
        setores_risk['area_risco_baixo_km2'] = 0
        setores_risk['pop_risco_alto'] = 0
        setores_risk['pop_risco_medio'] = 0
        setores_risk['pop_risco_baixo'] = 0
        setores_risk['pct_area_risco'] = 0
        setores_risk['pct_pop_risco'] = 0
        setores_risk['nivel_risco_predominante'] = 'Sem risco'
        
        # Para cada nível de risco
        for nivel in sorted(inundation_gdf['nivel_risco'].unique()):
            # Filtrar áreas para o nível atual
            risk_areas = inundation_gdf[inundation_gdf['nivel_risco'] == nivel]
            risk_union = unary_union(risk_areas.geometry)
            
            # Para cada setor, calcular a interseção
            for idx, setor in tqdm(setores_risk.iterrows(), total=len(setores_risk), 
                               desc=f"Processando áreas de risco {nivel}"):
                try:
                    if setor.geometry.intersects(risk_union):
                        # Calcular interseção
                        intersection = setor.geometry.intersection(risk_union)
                        
                        # Calcular área de interseção em km²
                        intersection_area = intersection.area / 1_000_000
                        
                        # Atualizar colunas específicas por nível
                        if nivel == 3:  # Risco Alto
                            setores_risk.at[idx, 'area_risco_alto_km2'] = intersection_area
                        elif nivel == 2:  # Risco Médio
                            setores_risk.at[idx, 'area_risco_medio_km2'] = intersection_area
                        elif nivel == 1:  # Risco Baixo
                            setores_risk.at[idx, 'area_risco_baixo_km2'] = intersection_area
                        
                        # Calcular percentual da área em risco
                        setor_area = setor.geometry.area / 1_000_000
                        pct_area = (intersection_area / setor_area) * 100
                        
                        # Estimar população afetada (proporcional à área)
                        pop_affected = setor[pop_col] * (intersection_area / setor_area)
                        
                        # Atualizar colunas específicas por nível
                        if nivel == 3:  # Risco Alto
                            setores_risk.at[idx, 'pop_risco_alto'] = pop_affected
                        elif nivel == 2:  # Risco Médio
                            setores_risk.at[idx, 'pop_risco_medio'] = pop_affected
                        elif nivel == 1:  # Risco Baixo
                            setores_risk.at[idx, 'pop_risco_baixo'] = pop_affected
                except Exception as e:
                    print(f"Erro ao processar setor {idx}: {e}")
        
        # Calcular totais e percentuais
        setores_risk['area_risco_total_km2'] = (
            setores_risk['area_risco_alto_km2'] + 
            setores_risk['area_risco_medio_km2'] + 
            setores_risk['area_risco_baixo_km2']
        )
        
        setores_risk['pop_risco_total'] = (
            setores_risk['pop_risco_alto'] + 
            setores_risk['pop_risco_medio'] + 
            setores_risk['pop_risco_baixo']
        )
        
        # Calcular área total de cada setor
        setores_risk['area_total_km2'] = setores_risk.geometry.area / 1_000_000
        
        # Calcular percentuais
        setores_risk['pct_area_risco'] = (setores_risk['area_risco_total_km2'] / setores_risk['area_total_km2']) * 100
        setores_risk['pct_pop_risco'] = (setores_risk['pop_risco_total'] / setores_risk[pop_col]) * 100
        
        # Determinar nível de risco predominante
        conditions = [
            setores_risk['pop_risco_alto'] > 0,
            setores_risk['pop_risco_medio'] > 0,
            setores_risk['pop_risco_baixo'] > 0
        ]
        choices = ['Risco Alto', 'Risco Médio', 'Risco Baixo']
        setores_risk['nivel_risco_predominante'] = np.select(conditions, choices, default='Sem risco')
        
        return setores_risk
    
    except Exception as e:
        print(f"Erro ao avaliar população afetada: {e}")
        return setores_gdf

# Avaliar população afetada
try:
    setores_risk = avaliar_populacao_afetada(setores, inundation_areas)
    
    # Calcular estatísticas de população
    pop_total = setores_risk['est_populacao'].sum()
    pop_risco_total = setores_risk['pop_risco_total'].sum()
    pop_risco_alto = setores_risk['pop_risco_alto'].sum()
    pop_risco_medio = setores_risk['pop_risco_medio'].sum()
    pop_risco_baixo = setores_risk['pop_risco_baixo'].sum()
    
    # Imprimir estatísticas
    print("\nEstatísticas de População em Risco:")
    print(f"População total: {pop_total:.0f} habitantes")
    print(f"População em áreas de risco: {pop_risco_total:.0f} habitantes ({pop_risco_total / pop_total * 100:.2f}%)")
    print(f"   - Risco Alto: {pop_risco_alto:.0f} habitantes ({pop_risco_alto / pop_total * 100:.2f}%)")
    print(f"   - Risco Médio: {pop_risco_medio:.0f} habitantes ({pop_risco_medio / pop_total * 100:.2f}%)")
    print(f"   - Risco Baixo: {pop_risco_baixo:.0f} habitantes ({pop_risco_baixo / pop_total * 100:.2f}%)")
    
    # Visualizar população em risco por setor
    fig, ax = plt.subplots(figsize=(16, 12))
    
    # Plotar setores coloridos por percentual de população em risco
    setores_risk.plot(
        column='pct_pop_risco',
        ax=ax,
        cmap='YlOrRd',
        legend=True,
        legend_kwds={'label': 'População em Risco (%)'},
        missing_kwds={'color': 'lightgray'},
        scheme='quantiles',
        k=5
    )
    
    # Plotar hidrografia sobre os setores
    trecho_drenagem.plot(ax=ax, color='blue', linewidth=1, alpha=0.7)
    curso_dagua.plot(ax=ax, color='darkblue', linewidth=1.5, alpha=0.7)
    
    # Configurar estilo
    ax.set_title('Percentual da População em Área de Risco por Setor Censitário', fontsize=16)
    ax.set_axis_off()
    plt.tight_layout()
    plt.show()
    
    # Gráfico de barras para os setores mais afetados
    top_affected = setores_risk.nlargest(10, 'pct_pop_risco')
    
    plt.figure(figsize=(14, 8))
    bars = plt.bar(range(len(top_affected)), top_affected['pct_pop_risco'], color='darkred')
    plt.xticks(range(len(top_affected)), top_affected.index, rotation=45, ha='right')
    plt.ylabel('População em Risco (%)')
    plt.title('10 Setores com Maior Percentual de População em Risco', fontsize=16)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Adicionar rótulos de valores
    for i, bar in enumerate(bars):
        plt.text(
            bar.get_x() + bar.get_width()/2,
            bar.get_height() + 1,
            f"{top_affected['pct_pop_risco'].iloc[i]:.1f}%",
            ha='center',
            fontsize=10
        )
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Erro ao avaliar ou visualizar população afetada: {e}")

# %% [markdown]
# ## 9. Construção de um Modelo de Risco Integrado
# Vamos construir um modelo integrado que combine informações de edificações, hidrografia e população.

# %%
def criar_modelo_risco_integrado(buildings_risk, setores_risk, inundation_areas=None):
    """
    Cria um modelo integrado de risco, combinando fatores de edificações e população.
    
    Args:
        buildings_risk: GeoDataFrame de edificações com informações de risco
        setores_risk: GeoDataFrame de setores com informações de risco
        inundation_areas: GeoDataFrame com áreas de inundação
        
    Returns:
        Dicionário com componentes do modelo integrado
    """
    try:
        # 1. Calcular densidade de edificações em risco por setor
        if buildings_risk is not None and setores_risk is not None:
            # Garantir que estão no mesmo CRS
            if buildings_risk.crs != setores_risk.crs:
                buildings_risk = buildings_risk.to_crs(setores_risk.crs)
            
            # Criar coluna para identificar edificações em risco
            buildings_risk['em_risco'] = buildings_risk['risco_inundacao'] > 0
            
            # Contagem espacial de edificações por setor
            buildings_count = gpd.sjoin(
                buildings_risk, 
                setores_risk[['geometry']], 
                how='left', 
                predicate='within'
            )
            
            # Agrupar por setor e contar edificações
            building_counts = buildings_count.groupby('index_right').agg({
                'em_risco': ['count', 'sum']
            })
            building_counts.columns = ['total_edificacoes', 'edificacoes_em_risco']
            building_counts = building_counts.reset_index()
            
            # Mesclar com setores
            setores_model = setores_risk.merge(
                building_counts, 
                left_index=True, 
                right_on='index_right', 
                how='left'
            )
            
            # Preencher NAs
            setores_model['total_edificacoes'] = setores_model['total_edificacoes'].fillna(0)
            setores_model['edificacoes_em_risco'] = setores_model['edificacoes_em_risco'].fillna(0)
            
            # Calcular densidade de edificações em risco
            setores_model['dens_edif_risco'] = setores_model['edificacoes_em_risco'] / setores_model['area_total_km2']
            
            # Calcular percentual de edificações em risco
            setores_model['pct_edif_risco'] = np.where(
                setores_model['total_edificacoes'] > 0,
                (setores_model['edificacoes_em_risco'] / setores_model['total_edificacoes']) * 100,
                0
            )
            
            # 2. Criar índice integrado de risco (0-100)
            # Fator 1: Percentual de área em risco (30%)
            # Fator 2: Percentual de população em risco (40%)
            # Fator 3: Percentual de edificações em risco (30%)
            
            # Normalizar cada fator para 0-100
            max_area_pct = setores_model['pct_area_risco'].max()
            max_pop_pct = setores_model['pct_pop_risco'].max()
            max_edif_pct = setores_model['pct_edif_risco'].max()
            
            if max_area_pct > 0:
                setores_model['area_risco_norm'] = (setores_model['pct_area_risco'] / max_area_pct) * 100
            else:
                setores_model['area_risco_norm'] = 0
                
            if max_pop_pct > 0:
                setores_model['pop_risco_norm'] = (setores_model['pct_pop_risco'] / max_pop_pct) * 100
            else:
                setores_model['pop_risco_norm'] = 0
                
            if max_edif_pct > 0:
                setores_model['edif_risco_norm'] = (setores_model['pct_edif_risco'] / max_edif_pct) * 100
            else:
                setores_model['edif_risco_norm'] = 0
            
            # Calcular índice integrado
            setores_model['indice_risco_integrado'] = (
                0.3 * setores_model['area_risco_norm'] + 
                0.4 * setores_model['pop_risco_norm'] + 
                0.3 * setores_model['edif_risco_norm']
            )
            
            # Categorizar o índice
            setores_model['categoria_risco'] = pd.cut(
                setores_model['indice_risco_integrado'],
                bins=[0, 20, 40, 60, 80, 100],
                labels=['Muito Baixo', 'Baixo', 'Médio', 'Alto', 'Muito Alto']
            )
            
            return {
                'setores_model': setores_model,
                'buildings_risk': buildings_risk,
                'inundation_areas': inundation_areas
            }
        else:
            print("Dados insuficientes para criar modelo integrado")
            return None
            
    except Exception as e:
        print(f"Erro ao criar modelo de risco integrado: {e}")
        return None

# Criar modelo integrado
try:
    integrated_model = criar_modelo_risco_integrado(buildings_risk, setores_risk, inundation_areas)
    
    if integrated_model:
        setores_model = integrated_model['setores_model']
        
        # Contabilizar setores por categoria de risco
        risk_counts = setores_model['categoria_risco'].value_counts()
        print("\nSetores por Categoria de Risco:")
        print(risk_counts)
        
        # Contabilizar população por categoria de risco
        pop_by_risk = setores_model.groupby('categoria_risco')['est_populacao'].sum()
        print("\nPopulação por Categoria de Risco:")
        print(pop_by_risk)
        
        # Visualizar índice de risco integrado
        fig, ax = plt.subplots(figsize=(16, 12))
        
        # Plotar setores coloridos por índice de risco
        setores_model.plot(
            column='indice_risco_integrado',
            ax=ax,
            cmap='YlOrRd',
            legend=True,
            legend_kwds={'label': 'Índice de Risco Integrado'},
            missing_kwds={'color': 'lightgray'},
            scheme='quantiles',
            k=5
        )
        
        # Plotar hidrografia
        trecho_drenagem.plot(ax=ax, color='blue', linewidth=1, alpha=0.7)
        curso_dagua.plot(ax=ax, color='darkblue', linewidth=1.5, alpha=0.7)
        
        # Configurar estilo
        ax.set_title('Índice Integrado de Risco de Inundação', fontsize=16)
        ax.set_axis_off()
        plt.tight_layout()
        plt.show()
        
        # Histograma do índice de risco
        plt.figure(figsize=(14, 8))
        plt.hist(setores_model['indice_risco_integrado'], bins=20, color='orangered', alpha=0.7)
        plt.title('Distribuição do Índice de Risco Integrado', fontsize=16)
        plt.xlabel('Índice de Risco')
        plt.ylabel('Número de Setores')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()
        
        # Gráfico de dispersão: População x Risco
        plt.figure(figsize=(14, 8))
        plt.scatter(
            setores_model['est_populacao'],
            setores_model['indice_risco_integrado'],
            c=setores_model['pct_area_risco'],
            cmap='YlOrRd',
            alpha=0.7,
            s=50
        )
        plt.colorbar(label='Área em Risco (%)')
        plt.title('Relação entre População e Risco de Inundação', fontsize=16)
        plt.xlabel('População Estimada')
        plt.ylabel('Índice de Risco Integrado')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()
        
    else:
        print("Não foi possível criar o modelo integrado de risco")
    
except Exception as e:
    print(f"Erro ao criar ou visualizar modelo integrado: {e}")

# %% [markdown]
# ## 10. Salvando Resultados e Conclusões

# %%
# Salvar resultados do modelo integrado
try:
    if 'integrated_model' in locals() and integrated_model:
        # Salvar setores com modelo integrado
        setores_output = os.path.join(DATA_DIR, "enriched_data/setores_risco_inundacao.gpkg")
        integrated_model['setores_model'].to_file(setores_output, driver="GPKG")
        print(f"Setores com modelo de risco salvos em: {setores_output}")
        
        # Salvar edificações com risco
        buildings_output = os.path.join(DATA_DIR, "enriched_data/buildings_risco_inundacao.gpkg")
        integrated_model['buildings_risk'].to_file(buildings_output, driver="GPKG")
        print(f"Edificações com risco de inundação salvas em: {buildings_output}")
        
        # Salvar áreas de inundação
        if inundation_areas is not None:
            inundation_output = os.path.join(DATA_DIR, "enriched_data/areas_inundacao.gpkg")
            inundation_areas.to_file(inundation_output, driver="GPKG")
            print(f"Áreas de inundação salvas em: {inundation_output}")
            
        # Criar e salvar relatório em JSON
        report = {
            "data_analise": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"),
            "estatisticas_edificacoes": {
                "total": len(buildings_risk),
                "em_risco": int(buildings_risk['risco_inundacao'].sum() > 0),
                "risco_alto": int((buildings_risk['risco_inundacao'] == 3).sum()),
                "risco_medio": int((buildings_risk['risco_inundacao'] == 2).sum()),
                "risco_baixo": int((buildings_risk['risco_inundacao'] == 1).sum()),
                "percentual_em_risco": float((buildings_risk['risco_inundacao'] > 0).mean() * 100)
            },
            "estatisticas_populacao": {
                "total": float(pop_total),
                "em_risco": float(pop_risco_total),
                "risco_alto": float(pop_risco_alto),
                "risco_medio": float(pop_risco_medio),
                "risco_baixo": float(pop_risco_baixo),
                "percentual_em_risco": float((pop_risco_total / pop_total) * 100)
            },
            "estatisticas_areas": {
                "total_km2": float(setores_model['area_total_km2'].sum()),
                "em_risco_km2": float(setores_model['area_risco_total_km2'].sum()),
                "risco_alto_km2": float(setores_model['area_risco_alto_km2'].sum()),
                "risco_medio_km2": float(setores_model['area_risco_medio_km2'].sum()),
                "risco_baixo_km2": float(setores_model['area_risco_baixo_km2'].sum()),
                "percentual_em_risco": float((setores_model['area_risco_total_km2'].sum() / setores_model['area_total_km2'].sum()) * 100)
            },
            "setores_risco": {
                "muito_alto": int((setores_model['categoria_risco'] == 'Muito Alto').sum()),
                "alto": int((setores_model['categoria_risco'] == 'Alto').sum()),
                "medio": int((setores_model['categoria_risco'] == 'Médio').sum()),
                "baixo": int((setores_model['categoria_risco'] == 'Baixo').sum()),
                "muito_baixo": int((setores_model['categoria_risco'] == 'Muito Baixo').sum())
            }
        }
        
        # Salvar relatório
        report_output = os.path.join(DATA_DIR, "analysis_reports/inundacao_report.json")
        with open(report_output, 'w', encoding='utf-8') as f:
            json.dump(report, f, ensure_ascii=False, indent=4)
        print(f"Relatório de análise salvo em: {report_output}")
        
except Exception as e:
    print(f"Erro ao salvar resultados: {e}")

# %% [markdown]
# ## Conclusões da Análise de Risco de Inundação
# 
# Este notebook realizou uma análise abrangente do risco de inundações, integrando dados hidrográficos, topográficos, edificações e setores censitários. Os principais resultados incluem:
# 
# 1. **Modelagem Hidrográfica**:
#    - Construção de um grafo representando a rede hidrográfica
#    - Análise de acumulação de fluxo para identificação de áreas críticas
#    - Utilização de ordem de Strahler para ponderação do potencial de inundação
# 
# 2. **Identificação de Áreas de Risco**:
#    - Desenvolvimento de um modelo de susceptibilidade a inundações baseado em:
#      - Elevação do terreno
#      - Declividade
#      - Distância a corpos d'água
#      - Acumulação de fluxo
#    - Delimitação de zonas com diferentes níveis de risco (alto, médio, baixo)
# 
# 3. **Avaliação de Edificações em Risco**:
#    - Quantificação de edificações em diferentes zonas de risco
#    - Análise de vulnerabilidade por tipo de edificação
#    - Identificação de áreas prioritárias para intervenção
# 
# 4. **Impacto Populacional**:
#    - Estimativa da população potencialmente afetada por inundações
#    - Identificação de setores críticos com alta densidade populacional em áreas de risco
#    - Categorização de setores por prioridade de ação
# 
# 5. **Modelo Integrado de Risco**:
#    - Desenvolvimento de um índice composto que combina:
#      - Percentual de área em risco
#      - Percentual de população em risco
#      - Percentual de edificações em risco
#    - Classificação de setores em categorias de risco para priorização de intervenções
# 
# Estes resultados podem subsidiar:
# - Planejamento urbano e ambiental
# - Elaboração de planos de contingência
# - Políticas de uso e ocupação do solo
# - Projetos de drenagem urbana
# - Sistemas de alerta precoce